In [ ]:
env: scGPT_2

# import

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

import pickle
import sys
import requests

from types import MethodType

In [2]:
import json
import os
import sys
import time
import copy
from pathlib import Path
from typing import Iterable, List, Tuple, Dict, Union, Optional
import warnings

import torch
import numpy as np
import matplotlib
from torch import nn
from torch.nn import functional as F
from torchtext.vocab import Vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)


sys.path.insert(0, "/data1/lichen/code/single_cell_perturbation/others/scGPT/")

import scgpt as scg
from scgpt.model import TransformerGenerator
from scgpt.loss import (
    masked_mse_loss,
    criterion_neg_log_bernoulli,
    masked_relative_error,
)
from scgpt.tokenizer import tokenize_batch, pad_batch, tokenize_and_pad_batch
from scgpt.tokenizer.gene_tokenizer import GeneVocab
from scgpt.utils import set_seed, map_raw_id_to_vocab_id

# matplotlib.rcParams["savefig.transparent"] = False
warnings.filterwarnings("ignore")

set_seed(42)

In [3]:
from torch_geometric.loader import DataLoader
from gears import PertData, GEARS
from gears.inference import compute_metrics, deeper_analysis, non_dropout_analysis
from gears.utils import create_cell_graph_dataset_for_prediction

import importlib
from scperturb import *
import anndata as ad

In [4]:
sys.path.append("/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/")

In [5]:
import v1
from v1.utils import *
from v1.dataloader import *

# initial

In [25]:
dataset_pert_dict = {
    'CAR_T': ['PDCD1'],
    'blood': ['GATA1', 'SPI1'],
    'OSKM': [['SOX2',
         'POU5F1',
         'KLF4',
         'MYC']],
    'ADM': ['PTF1A']
}
dataset_celltype_dict = {
    'CAR_T': 'Tex', 
    'blood': 'LMPP',
    'OSKM': 'Fibroblast-like',
    'ADM': 'Acinar'
}

dataset_dire_dict = {
    'CAR_T': 'down', 
    'blood': 'down',
    'OSKM': 'up',
    'ADM': 'down'
}


datasets = list(dataset_pert_dict.keys())

In [7]:
#=========================basic parameters

# - init scGPT para
# settings for data prcocessing
pad_token = "<pad>"
special_tokens = [pad_token, "<cls>", "<eoc>"]
pad_value = 0  # for padding values
pert_pad_id = 2

n_hvg = 0  # number of highly variable genes
include_zero_gene = "all"  # include zero expr genes in training input, "all", "batch-wise", "row-wise", or False
# max_seq_len = 1536
max_seq_len = 6000


# settings for training
MLM = True  # whether to use masked language modeling, currently it is always on.
CLS = False  # celltype classification objective
CCE = False  # Contrastive cell embedding objective
MVC = False  # Masked value prediction for cell embedding
ECS = False  # Elastic cell similarity objective
cell_emb_style = "cls"
mvc_decoder_style = "inner product, detach"
amp = True
load_model = "/data1/lichen/code/single_cell_perturbation/others/scGPT/save/scGPT_human"

load_param_prefixs = [
    "encoder",
    "value_encoder",
    "transformer_encoder",
]

# settings for optimizer
lr = 1e-4  # or 1e-4
batch_size = 64
eval_batch_size = 64
epochs = 15
schedule_interval = 1
early_stop = 5

# settings for the model
embsize = 512  # embedding dimension
d_hid = 512  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 12  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 8  # number of heads in nn.MultiheadAttention
n_layers_cls = 3
dropout = 0.2  # dropout probability
use_fast_transformer = True  # whether to use fast transformer

# logging
log_interval = 100

# dataset and evaluation choices
data_name = "adamson"
split = "simulation"
if data_name == "norman":
    perts_to_plot = ["SAMD1+ZBTB1"]
elif data_name == "adamson":
    perts_to_plot = ["KCTD16+ctrl"]

# - 恢复默认参数
# plt.rcdefaults()

#==========================set GPU
# - multi gpu para
n_gpu = 3
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device_ids = [
                # 0,
               1, 
               # 2, 
               # 3,
               ]

# - training epoch
epochs = 20
max_seq_len = 6000
early_stop = 20
save_flag = True




#===========================set vocab
# - get the model 
model_dir = Path(load_model)


# - get the vocab
model_config_file = model_dir / "args.json"
model_file = model_dir / "best_model.pt"
vocab_file = model_dir / "vocab.json"

vocab = GeneVocab.from_file(vocab_file)

# token2idx = json.load(vocab_file)
with vocab_file.open("r") as f:
    token2idx = json.load(f)
print('length of the token is :',len(token2idx))


# - add token to the vocab
for s in special_tokens:
    if s not in vocab:
        vocab.append_token(s)

length of the token is : 60697


In [23]:
#=====================================load scGPT model
# - load the model
with open(model_config_file, "r") as f:
    model_configs = json.load(f)
print(
    f"Resume model from {model_file}, the model args will override the "
    f"config {model_config_file}."
)
embsize = model_configs["embsize"]
nhead = model_configs["nheads"]
d_hid = model_configs["d_hid"]
nlayers = model_configs["nlayers"]
n_layers_cls = model_configs["n_layers_cls"]


# - intial the model

ntokens = len(vocab)  # size of vocabulary
model = TransformerGenerator(
    ntokens,
    embsize,
    nhead,
    d_hid,
    nlayers,
    nlayers_cls=n_layers_cls,
    n_cls=1,
    vocab=vocab,
    dropout=dropout,
    pad_token=pad_token,
    pad_value=pad_value,
    pert_pad_id=pert_pad_id,
    do_mvc=MVC,
    cell_emb_style=cell_emb_style,
    mvc_decoder_style=mvc_decoder_style,
    use_fast_transformer=use_fast_transformer,
)

# - load the model
if load_param_prefixs is not None and load_model is not None:

    # -- load out weight
    # model_file = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/volc_result/scGPT_pretrain/down/model_best.pt'
    model_file = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/volc_result/scGPT_pretrain/up/model_best.pt'
    # model_file = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/volc_result/scGPT_pretrain_initial/down/model_best.pt'

    # 加载保存的模型权重
    saved_state_dict = torch.load(model_file)
    from collections import OrderedDict
    # 创建一个新的字典来存储修改后的权重
    new_state_dict = OrderedDict()

    # 修改键名称
    for key, value in saved_state_dict.items():
        new_key = key.replace('module.', '')  # 移除'module.'前缀
        new_state_dict[new_key] = value

    # 加载修改后的权重
    model.load_state_dict(new_state_dict)

# - add the parallel
model = torch.nn.DataParallel(model, device_ids=device_ids)

# - put model on device
model.to(device)

best_model = copy.deepcopy(model)

Resume model from /nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/volc_result/scGPT_pretrain/down/model_best.pt, the model args will override the config /data1/lichen/code/single_cell_perturbation/others/scGPT/save/scGPT_human/args.json.
Using simple batchnorm instead of domain specific batchnorm


# scGPT跑所有datasets

In [11]:
for dataset in datasets[:]:
    # dataset = 'OSKM'
    dataset = 'blood'
    print('='*20, dataset, '='*20)
    save_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/real_case/data'
    adata_rna = sc.read(os.path.join(save_dir, dataset, 'adata_ctrl.h5ad'))
    if not isinstance(adata_rna.X, np.ndarray):
        adata_rna.X = adata_rna.X.toarray()
    adata = adata_rna.copy()

    # - init adata
    adata_rna.var["gene_name"] = list(adata_rna.var_names)
    adata_rna.var["id_in_vocab"] = [
        1 if gene in vocab else -1 for gene in adata_rna.var["gene_name"]
    ]
    gene_ids_in_vocab = np.array(adata_rna.var["id_in_vocab"])
    print(
        f"match {np.sum(gene_ids_in_vocab >= 0)}/{len(gene_ids_in_vocab)} genes "
        f"in vocabulary of size {len(vocab)}."
    )
    genes = adata_rna.var["gene_name"].tolist()

    # - get token of input genes
    vocab.set_default_index(vocab["<pad>"])
    gene_ids = np.array(
        [vocab[gene] if gene in vocab else vocab["<pad>"] for gene in genes], dtype=int
    )
    n_genes = len(genes)

    for pert in tqdm(dataset_pert_dict[dataset]):
        # pert = ['SOX2','POU5F1','KLF4','MYC']
        print('*'*20, pert)
        # - use for multiple perts
        if isinstance(pert, str):
            pert_combo = [pert]
        else:
            pert_combo = pert
        # - prepare the test_loader
        Xs = adata_rna.X
        var_names = list(adata_rna.var_names)

        cell_graphs = []
        for X in Xs:
            # - pert_flags for multi perts
            pert_flags = torch.zeros(Xs.shape[1])
            for tmp_pert in pert_combo:
                if tmp_pert not in var_names:
                    continue
                else:
                    pert_flags[var_names.index(tmp_pert)] = 1
            tmp_Data = Data(x=torch.Tensor(X.reshape(1,-1)),
                        pert_flags=pert_flags.reshape(1,-1))
            cell_graphs.append(tmp_Data)

        test_loader = DataLoader(cell_graphs,
                                batch_size=32, shuffle=False)

        pred = []
        for itr, batch in enumerate(test_loader):
            batch.to(device)
            with torch.no_grad():
                p = pred_perturb_new(best_model, batch, include_zero_gene, gene_ids=gene_ids)
                pred.extend(p.cpu())
        pred = torch.stack(pred)
        pred = pred.detach().cpu().numpy()

        pert_prefix = '_'.join(pert_combo)
        # - create adata_pert
        adata_pert = adata_rna.copy()
        adata_pert.X = pred
        adata_pert.obs_names = [i+f'_{pert_prefix}' for i in adata_pert.obs_names]

        # - adata_ctrl
        adata_ctrl = adata_rna.copy()

        # tmp_dir = f'/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/real_case/data/{dataset}'
        tmp_dir = f'/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/real_case/result/{dataset}'
        save_prefix = f'scGPT/{pert_prefix}' # use result of K562 to do the direct transfer
        os.makedirs(os.path.join(tmp_dir, save_prefix), exist_ok=True)
        adata_pert.write(os.path.join(tmp_dir, save_prefix, 'adata_pert.h5ad'))

        break

    break


==================== blood ====================
match 1205/1206 genes in vocabulary of size 60697.


  0%|          | 0/2 [00:00<?, ?it/s]

******************** GATA1


  0%|          | 0/2 [00:27<?, ?it/s]


# scGPT跑所有datasets - 使用minus计算出来delta，然后尝试绘制transition score

In [37]:
adata_mode = 'non_minus'
# minus: save adata as the minus delta
# non_minus: save adata, add the minus delta on the original gene exp

for dataset in datasets[:]:
    # dataset = 'OSKM'
    # dataset = 'blood'
    # dataset = 'CAR_T'
    print('='*20, dataset, '='*20)
    save_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/real_case/data'
    adata_rna = sc.read(os.path.join(save_dir, dataset, 'adata_ctrl_v2.h5ad'))
    if not isinstance(adata_rna.X, np.ndarray):
        adata_rna.X = adata_rna.X.toarray()
    adata = adata_rna.copy()

    # - init adata
    adata_rna.var["gene_name"] = list(adata_rna.var_names)
    adata_rna.var["id_in_vocab"] = [
        1 if gene in vocab else -1 for gene in adata_rna.var["gene_name"]
    ]
    gene_ids_in_vocab = np.array(adata_rna.var["id_in_vocab"])
    print(
        f"match {np.sum(gene_ids_in_vocab >= 0)}/{len(gene_ids_in_vocab)} genes "
        f"in vocabulary of size {len(vocab)}."
    )
    genes = adata_rna.var["gene_name"].tolist()

    # - get token of input genes
    vocab.set_default_index(vocab["<pad>"])
    gene_ids = np.array(
        [vocab[gene] if gene in vocab else vocab["<pad>"] for gene in genes], dtype=int
    )
    n_genes = len(genes)


    #################################################### load model, OSKM for up model
    model = TransformerGenerator(
        ntokens,
        embsize,
        nhead,
        d_hid,
        nlayers,
        nlayers_cls=n_layers_cls,
        n_cls=1,
        vocab=vocab,
        dropout=dropout,
        pad_token=pad_token,
        pad_value=pad_value,
        pert_pad_id=pert_pad_id,
        do_mvc=MVC,
        cell_emb_style=cell_emb_style,
        mvc_decoder_style=mvc_decoder_style,
        use_fast_transformer=use_fast_transformer,
    )
    # - load the model
    if load_param_prefixs is not None and load_model is not None:

        # -- load out weight
        # model_file = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/volc_result/scGPT_pretrain/down/model_best.pt'
        # model_file = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/volc_result/scGPT_pretrain/up/model_best.pt'
        # model_file = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/volc_result/scGPT_pretrain_initial/down/model_best.pt'

        if dataset_dire_dict[dataset] == 'down':
            model_file = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/volc_result/scGPT_pretrain/down/model_best.pt'
        if dataset_dire_dict[dataset] == 'up':
            model_file = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/volc_result/scGPT_pretrain/up/model_best.pt'



        # 加载保存的模型权重
        saved_state_dict = torch.load(model_file)
        from collections import OrderedDict
        # 创建一个新的字典来存储修改后的权重
        new_state_dict = OrderedDict()

        # 修改键名称
        for key, value in saved_state_dict.items():
            new_key = key.replace('module.', '')  # 移除'module.'前缀
            new_state_dict[new_key] = value

        # 加载修改后的权重
        model.load_state_dict(new_state_dict)

    # - add the parallel
    model = torch.nn.DataParallel(model, device_ids=device_ids)

    # - put model on device
    model.to(device)

    best_model = copy.deepcopy(model)

    #################################################### run the inference
    for pert in tqdm(dataset_pert_dict[dataset]):
        # pert = 'GATA1'
        # pert = ['SOX2','POU5F1','KLF4','MYC']
        # pert = 'SPI1'
        # pert = 'PDCD1'
        print('*'*20, pert)
        # - use for multiple perts
        if isinstance(pert, str):
            pert_combo = [pert]
        else:
            pert_combo = pert
        # - prepare the test_loader
        Xs = adata_rna.X
        var_names = list(adata_rna.var_names)

        #================================= get ctrl output
        cell_graphs = []
        for X in Xs:
            # - pert_flags for multi perts
            pert_flags = torch.zeros(Xs.shape[1])
            for tmp_pert in pert_combo:
                if tmp_pert not in var_names:
                    continue
                else:
                    pert_flags[var_names.index(tmp_pert)] = 0
            tmp_Data = Data(x=torch.Tensor(X.reshape(1,-1)),
                        pert_flags=pert_flags.reshape(1,-1))
            cell_graphs.append(tmp_Data)

        test_loader = DataLoader(cell_graphs,
                                batch_size=32, shuffle=False)

        pred = []
        for itr, batch in enumerate(test_loader):
            batch.to(device)
            with torch.no_grad():
                p = pred_perturb_new(best_model, batch, include_zero_gene, gene_ids=gene_ids)
                pred.extend(p.cpu())
        pred = torch.stack(pred)
        pred = pred.detach().cpu().numpy()
        pred_ctrl = pred.copy()

        #================================= get pert output
        cell_graphs = []
        for X in Xs:
            # - pert_flags for multi perts
            pert_flags = torch.zeros(Xs.shape[1])
            for tmp_pert in pert_combo:
                if tmp_pert not in var_names:
                    continue
                else:
                    pert_flags[var_names.index(tmp_pert)] = 1
            tmp_Data = Data(x=torch.Tensor(X.reshape(1,-1)),
                        pert_flags=pert_flags.reshape(1,-1))
            cell_graphs.append(tmp_Data)

        test_loader = DataLoader(cell_graphs,
                                batch_size=32, shuffle=False)

        pred = []
        for itr, batch in enumerate(test_loader):
            batch.to(device)
            with torch.no_grad():
                p = pred_perturb_new(best_model, batch, include_zero_gene, gene_ids=gene_ids)
                pred.extend(p.cpu())
        pred = torch.stack(pred)
        pred = pred.detach().cpu().numpy()
        pred_pert = pred.copy()

        pert_prefix = '_'.join(pert_combo)

        tmp_dir = f'/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/real_case/result/{dataset}'
        save_prefix = f'scGPT/{pert_prefix}' # use result of K562 to do the direct transfer
        os.makedirs(os.path.join(tmp_dir, save_prefix), exist_ok=True)


        adata_pert = adata_rna.copy()

        if adata_mode == 'minus':
            adata_pert.X = pred_pert - pred_ctrl
            adata_pert.obs_names = [i+f'_{pert_prefix}' for i in adata_pert.obs_names]
            adata_pert.write(os.path.join(tmp_dir, save_prefix, 'adata_pert_minus.h5ad'))

        elif adata_mode == 'non_minus':
            adata_pert.X += pred_pert - pred_ctrl
            adata_pert.obs_names = [i+f'_{pert_prefix}' for i in adata_pert.obs_names]
            adata_pert.write(os.path.join(tmp_dir, save_prefix, 'adata_pert.h5ad'))
        else:
            raise ValueError()

    #     break
    # break


==================== CAR_T ====================
match 2548/2647 genes in vocabulary of size 60697.
Using simple batchnorm instead of domain specific batchnorm


  0%|          | 0/1 [00:00<?, ?it/s]

******************** PDCD1


100%|██████████| 1/1 [00:13<00:00, 13.38s/it]


==================== blood ====================
match 1205/1206 genes in vocabulary of size 60697.
Using simple batchnorm instead of domain specific batchnorm


...

100%|██████████| 1/1 [01:38<00:00, 98.72s/it]


==================== ADM ====================
match 4511/4880 genes in vocabulary of size 60697.
Using simple batchnorm instead of domain specific batchnorm


  0%|          | 0/1 [00:00<?, ?it/s]

******************** PTF1A


100%|██████████| 1/1 [00:21<00:00, 21.29s/it]


In [33]:
adata_pert = adata_rna.copy()

# if adata_mode == 'minus':
adata_pert.X = pred_pert - pred_ctrl
adata_pert.obs_names = [i+f'_{pert_prefix}' for i in adata_pert.obs_names]
adata_pert.write(os.path.join(tmp_dir, save_prefix, 'adata_pert_minus.h5ad'))

In [30]:
adata_pert.X

array([[ 6.95647061e-01,  6.59179688e-03,  5.61523438e-03, ...,
         6.91984951e-01,  7.67424405e-01, -3.66210938e-04],
       [ 1.27201176e+00,  6.10351562e-03,  5.85937500e-03, ...,
         8.19480658e-01,  1.33963871e+00,  1.58691406e-03],
       [-2.92968750e-03,  5.85937500e-03,  4.88281250e-03, ...,
         1.22070312e-03,  1.31645989e+00,  1.04463267e+00],
       ...,
       [-1.46484375e-03,  6.83593750e-03,  5.37109375e-03, ...,
         2.19726562e-03,  3.84730864e+00,  1.95312500e-03],
       [ 6.02443039e-01,  6.34765625e-03,  4.63867188e-03, ...,
         7.32421875e-04,  2.88891983e+00,  1.95312500e-03],
       [-3.66210938e-03,  6.59179688e-03,  5.12695312e-03, ...,
         7.32421875e-04,  1.27215433e+00,  8.54492188e-04]])

In [31]:
pred_pert - pred_ctrl

array([[ 0.00439453,  0.0065918 ,  0.00561523, ...,  0.00073242,
         0.07617188, -0.00036621],
       [ 0.00952148,  0.00610352,  0.00585938, ...,  0.00097656,
         0.07714844,  0.00158691],
       [-0.00292969,  0.00585938,  0.00488281, ...,  0.0012207 ,
         0.07714844,  0.00085449],
       ...,
       [-0.00146484,  0.00683594,  0.00537109, ...,  0.00219727,
         0.10546875,  0.00195312],
       [ 0.00610352,  0.00634766,  0.00463867, ...,  0.00073242,
         0.0859375 ,  0.00195312],
       [-0.00366211,  0.0065918 ,  0.00512695, ...,  0.00073242,
         0.07617188,  0.00085449]], dtype=float32)

In [32]:
adata_rna.X

array([[0.69125253, 0.        , 0.        , ..., 0.69125253, 0.69125253,
        0.        ],
       [1.26249027, 0.        , 0.        , ..., 0.8185041 , 1.26249027,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.23931146,
        1.04377818],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 3.74183989,
        0.        ],
       [0.59633952, 0.        , 0.        , ..., 0.        , 2.80298233,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.19598246,
        0.        ]])

# debug

In [14]:
adata_pert.X

array([[ 0.00439453,  0.0065918 ,  0.00561523, ...,  0.00073242,
         0.07617188, -0.00036621],
       [ 0.00952148,  0.00610352,  0.00585938, ...,  0.00097656,
         0.07714844,  0.00158691],
       [-0.00292969,  0.00585938,  0.00488281, ...,  0.0012207 ,
         0.07714844,  0.00085449],
       ...,
       [-0.00146484,  0.00683594,  0.00537109, ...,  0.00219727,
         0.10546875,  0.00195312],
       [ 0.00610352,  0.00634766,  0.00463867, ...,  0.00073242,
         0.0859375 ,  0.00195312],
       [-0.00366211,  0.0065918 ,  0.00512695, ...,  0.00073242,
         0.07617188,  0.00085449]], dtype=float32)

In [12]:
adata_pert

AnnData object with n_obs × n_vars = 7073 × 1206
    obs: 'cid', 'seq_tech', 'donor_ID', 'donor_gender', 'donor_age', 'donor_status', 'original_name', 'organ', 'region', 'subregion', 'sample_status', 'treatment', 'ethnicity', 'cell_type', 'cell_id', 'study_id', 'age_bin', 'celltype', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'gene_name', 'id_in_vocab'
    uns: 'hvg'

# function

In [10]:
def pred_perturb_new(
    model,
    batch_data,
    include_zero_gene="batch-wise",
    gene_ids=None,
    amp=True,
):
    """
    Args:
        batch_data: a dictionary of input data with keys.

    Returns:
        output Tensor of shape [N, seq_len]
    """
    model.eval()
    device = next(model.parameters()).device
    batch_data.to(device)
    batch_size = len(batch_data.x)
    x: torch.Tensor = batch_data.x
    # ori_gene_values = x[:, 0].view(batch_size, n_genes)
    ori_gene_values = x
    # pert_flags = x[:, 1].long().view(batch_size, n_genes)
    pert_flags = batch_data.pert_flags.long()

    if include_zero_gene in ["all", "batch-wise"]:
        assert gene_ids is not None
        if include_zero_gene == "all":
            input_gene_ids = torch.arange(ori_gene_values.size(1), device=device)
        else:  # batch-wise
            input_gene_ids = (
                ori_gene_values.nonzero()[:, 1].flatten().unique().sort()[0]
            )
        input_values = ori_gene_values[:, input_gene_ids]
        input_pert_flags = pert_flags[:, input_gene_ids]

        mapped_input_gene_ids = map_raw_id_to_vocab_id(input_gene_ids, gene_ids)
        mapped_input_gene_ids = mapped_input_gene_ids.repeat(batch_size, 1)

        src_key_padding_mask = torch.zeros_like(
            input_values, dtype=torch.bool, device=device
        )
        with torch.cuda.amp.autocast(enabled=amp):
            output_dict = model(
                mapped_input_gene_ids,
                input_values,
                input_pert_flags,
                src_key_padding_mask=src_key_padding_mask,
                CLS=False,
                CCE=False,
                MVC=False,
                ECS=False,
                do_sample=True,
            )
        output_values = output_dict["mlm_output"].float()
        pred_gene_values = torch.zeros_like(ori_gene_values)
        pred_gene_values[:, input_gene_ids] = output_values
    return pred_gene_values